In [26]:
# -*- coding: utf-8 -*-
"""
File: wsh_webquery_download.py 
Parses HTTPResponse objects returned from webservice query

Created on Tues Sep  7 11:49:53 2021
Last modified: 5/5/22

@author: clj585
"""

###################################################
# Loads packages 
###################################################

import urllib.request
import xmltodict
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time

# Define path here 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/"

###################################################
# Send Webservice Query & pull data 
###################################################

"""
Function: formURL

Returns URL generated for WSH Enchilada webservice query 

@params: 
   stock_sym_list: list of strings of stock symbols
   class: string or ideally only 1 string
   from_dt: start date formatted as string
   to_dt: end date of query as string
"""
def formURL(stock_sym_list, evtclass, from_dt, to_dt):
    
    home = "https://enchilada.wallstreethorizon.com/webservice6.asp"
    cust_id = "?c=NORTHWESTERN"
    passw = "&p=4IQD42P0"
    
    stock_syms = "&stock_symbols="+",".join(stock_sym_list)
    classes = "&classes="+evtclass
    
    fromdt = "&from=" + from_dt
    todt = "&to=" + to_dt
    
    v = "&v=2"
    opts = "&o=EVENTS"
    
    url_str = (home + cust_id + passw + 
               stock_syms + 
               classes + fromdt + todt + 
               v + opts)
    return url_str

"""
Function: perDelta

Returns an iterable with all the starting dates of time intervals.
Breaks the history time window up into 9-day windows. 

@params: 
   start: string formatted date
   end:also string formatted end date
   delta: time-skip needs to be in days
"""
def perDelta(start, end, delta):
    start = datetime.strptime(start, "%m/%d/%y")
    end = datetime.strptime(end, "%m/%d/%y")
    curr = start
    while curr < end:
        yield curr
        curr += delta

###################################################
# Process EPS announce date in general framework
###################################################

# PARAMETERS AND LISTS CAN BE MODIFIED HERE # 
stock_sym_list = ["*"]
# Entire list of desired event class types
#class_list = ["BYBK", "CC", "DB", 
#              "DIV", "DIVSR", "ED", "EPS", 
#              "FDIV", "FQ", "OPTION", 
#              "QE", "SEC"]
class_list = ["EPS"]

# ToDo Will: UPDATE BASED ON LOGS 
hist_from_dt = (datetime.today()-relativedelta(years=1)).strftime(
    "%m/%d/%y")
hist_to_dt = (datetime.today()).strftime("%m/%d/%y")
    
for evt in class_list:          # if stock wildcard only 1 evt at a time

    # Sleep condition avoid timeout
    if(class_list.index(evt)%4 == 0 and class_list.index(evt)!=0): 
        time.sleep(30)    
    
    # Generate 9-day intervals for query window
    for start_dt in perDelta(hist_from_dt, 
                             #(datetime.today()-relativedelta(years=1)+relativedelta(days=3)).strftime("%m/%d/%y"),
                             hist_to_dt,
                             timedelta(days=1)): 
        
        # Formatting as strings 
        from_dt = start_dt.strftime("%m/%d/%y")
        to_dt = (start_dt + relativedelta(days=1)).strftime(
            "%m/%d/%y")
        
        # Formulate URL for webservice query here 
        url3 = formURL(stock_sym_list, evt, 
                       from_dt, to_dt)
        print(url3)

        # Send, process request and save tree structure 
        rsp = urllib.request.urlopen(url3)
        string = rsp.read().decode('utf-8')
    
        # Conversion to dictionary 
        dictdata = xmltodict.parse(string)

        # For each dictionary list (keyed by event class) get data 
        for key in dictdata["webservice6"].keys():
            dataf = pd.DataFrame()                  # placeholder
            
            # Check if OrderedDict or list
            if(type(dictdata["webservice6"][key]) != list):
                dct = dictdata["webservice6"][key]
                
                # Save to data frame and reformat
                d1 = pd.DataFrame(dct.items(), columns=None)
                dataf = pd.concat([dataf, d1])
                
            else:
                for i in range(len(dictdata["webservice6"][key])):
                
                    # Number of dictionaries equal to number of event id's 
                    dct = dictdata["webservice6"][key][i]
                
                    # Save to data frame and reformat
                    d1 = pd.DataFrame(dct.items(), columns=None)
                    d1 = d1.T
                    d1.rename(columns=d1.iloc[0], inplace = True)
                    d1.drop(d1.index[0], inplace = True)
                
                    # Extra column processing for actual earnings data
                    #if(key == "eps"):
                    #    d1["ea_datetime"] = pd.to_datetime(
                    #        d1["announce_datetime"])
                    #    d1["ea_date"] = d1["ea_datetime"].dt.strftime(
                    #        '%m/%d/%y')
                    #    d1["ea_time"] = d1["ea_datetime"].dt.strftime(
                    #        '%H:%M')
                        
                    dataf = pd.concat([dataf, d1])
            
            if(key != "error"):                     # deal with empty query
                dt_lbl = start_dt.strftime("%m-%d-%y")
                print(onedrive_pth + f"{key}_webserv{dt_lbl}.parquet")
                # ToDo Will: UPDATE THIS PATH
                #dataf.to_parquet(
                #    onedrive_pth + f"/{key}_webserv{dt_lbl}.parquet"
                #)
                #dataf.to_parquet(
                    #f"C://Users/clj585/Downloads/{key}_webserv{dt_lbl}.parquet", 
                    #index=None)
            else:
                print("no data in this date range for ", evt, ", ",
                      from_dt, " to ", to_dt, "! :)")
                
        # Process the data frame here 
        print(dataf.columns)
        dataf[['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
               'announce_datetime', 'quarter', 'quarter_end_date',
               'currency', 'amount_oc', 'purl', 'created', 'updated',
               'prelim_earnings_date', 'return_time', 'prelim_amount_from',
               'prelim_amount_to', 'prelim_earnings_link']] = dataf[['event_id', 'company_id', 'stock_symbol', 
                                                                     'isin', 'company_name',
                                                                     'announce_datetime', 'quarter', 
                                                                     'quarter_end_date',
                                                                     'currency', 'amount_oc', 'purl', 
                                                                     'created', 'updated',
                                                                     'prelim_earnings_date', 'return_time', 
                                                                     'prelim_amount_from',
                                                                     'prelim_amount_to', 
                                                                     'prelim_earnings_link']].astype("string")
        
        # Datetime columns manipulate 
        dataf['fiscal_year'] = dataf['fiscal_year'].astype(int)
        dataf['prelim_earnings_date'] = pd.to_datetime(dataf['prior_earnings_date'])
        dataf['announce_datetime'] = pd.to_datetime(dataf['announce_datetime'])
        dataf['quarter_end_date'] = pd.to_datetime(dataf['quarter_end_date'])
        dataf['return_time'] = pd.to_datetime(dataf['return_time'])
        dataf['updated'] = pd.to_datetime(dataf['updated'])
        dataf['created'] = pd.to_datetime(dataf['created'])
        
        # Create asofdate (date) column
        dataf['date'] = start_dt
        dataf['date'] = pd.to_datetime(dataf['date'])
        
        # Save to parquet file 
        print(onedrive_pth + f"temp_{key}/{key}_webserv{dt_lbl}.parquet")    
        #dataf.to_parquet(onedrive_pth + 
        #                 f"/{key}_webserv{dt_lbl}.parquet")

https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=05/06/21&to=05/07/21&v=2&o=EVENTS
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/eps_webserv05-06-21.parquet
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'prelim_earnings_date', 'return_time', 'prelim_amount_from',
       'prelim_amount_to', 'prelim_earnings_link'],
      dtype='object')
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/eps_webserv05-06-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=05/07/21&to=05/08/21&v=2&o=EVENTS
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/eps_webserv05-07-21.parquet
Index(['event_id', 'company_id', 'stock_symbol', 'i

In [38]:
# -*- coding: utf-8 -*-
"""
File: wsh_webquery_DB_download.py 
Parses HTTPResponse objects returned from webservice query

Created on Tues Sep  7 11:49:53 2021
Last modified: 5/5/22

@author: clj585
"""

###################################################
# Loads packages 
###################################################

import urllib.request
import xmltodict
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time

# Define path here 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/"

###################################################
# Send Webservice Query & pull data 
###################################################

"""
Function: formURL

Returns URL generated for WSH Enchilada webservice query 

@params: 
   stock_sym_list: list of strings of stock symbols
   class: string or ideally only 1 string
   from_dt: start date formatted as string
   to_dt: end date of query as string
"""
def formURL(stock_sym_list, evtclass, from_dt, to_dt):
    
    home = "https://enchilada.wallstreethorizon.com/webservice6.asp"
    cust_id = "?c=NORTHWESTERN"
    passw = "&p=4IQD42P0"
    
    stock_syms = "&stock_symbols="+",".join(stock_sym_list)
    classes = "&classes="+evtclass
    
    fromdt = "&from=" + from_dt
    todt = "&to=" + to_dt
    
    v = "&v=2"
    opts = "&o=EVENTS"
    
    url_str = (home + cust_id + passw + 
               stock_syms + 
               classes + fromdt + todt + 
               v + opts)
    return url_str

"""
Function: perDelta

Returns an iterable with all the starting dates of time intervals.
Breaks the history time window up into 9-day windows. 

@params: 
   start: string formatted date
   end:also string formatted end date
   delta: time-skip needs to be in days
"""
def perDelta(start, end, delta):
    start = datetime.strptime(start, "%m/%d/%y")
    end = datetime.strptime(end, "%m/%d/%y")
    curr = start
    while curr < end:
        yield curr
        curr += delta

###################################################
# Process EPS announce date in general framework
###################################################

# PARAMETERS AND LISTS CAN BE MODIFIED HERE # 
stock_sym_list = ["*"]
# Entire list of desired event class types
class_list = ['EPS']

# ToDo Will: UPDATE BASED ON LOGS 
hist_from_dt = (datetime.today()-relativedelta(years=1)).strftime(
    "%m/%d/%y")
hist_to_dt = (datetime.today()).strftime("%m/%d/%y")
    
for evt in class_list:          # if stock wildcard only 1 evt at a time

    # Sleep condition avoid timeout
    if(class_list.index(evt)%4 == 0 and class_list.index(evt)!=0): 
        time.sleep(30)    
    
    # Generate 9-day intervals for query window
    for start_dt in perDelta(hist_from_dt, 
                             #(datetime.today()-relativedelta(years=1)+relativedelta(days=10)).strftime("%m/%d/%y"),
                             #(datetime.today()-relativedelta(days=1)).strftime("%m/%d/%y"),
                             hist_to_dt,
                             timedelta(days=1)): 
        
        # Formatting as strings 
        from_dt = start_dt.strftime("%m/%d/%y")
        to_dt = (start_dt + relativedelta(days=1)).strftime(
            "%m/%d/%y")
        
        # Formulate URL for webservice query here 
        url3 = formURL(stock_sym_list, evt, 
                       from_dt, to_dt)
        print(url3)

        # Send, process request and save tree structure 
        rsp = urllib.request.urlopen(url3)
        string = rsp.read().decode('utf-8')
    
        # Conversion to dictionary 
        dictdata = xmltodict.parse(string)

        # For each dictionary list (keyed by event class) get data 
        for key in dictdata["webservice6"].keys():
            dataf = pd.DataFrame()                  # placeholder
            
            if key != "error":
                # Check if OrderedDict or list
                if(type(dictdata["webservice6"][key]) != list):
                    print("Are you here?")
                    dct = dictdata["webservice6"][key]
                
                    # Save to data frame and reformat
                    d1 = pd.DataFrame(dct.items(), columns=None)
                    d1 = d1.T
                    d1.rename(columns=d1.iloc[0], inplace = True)
                    d1.drop(d1.index[0], inplace = True)
                    dataf = pd.concat([dataf, d1])
                
                else:
                    print("It will be a list!")
                    for i in range(len(dictdata["webservice6"][key])):
                
                        # Number of dictionaries equal to number of event id's 
                        dct = dictdata["webservice6"][key][i]
                
                        # Save to data frame and reformat
                        d1 = pd.DataFrame(dct.items(), columns=None)
                        d1 = d1.T
                        d1.rename(columns=d1.iloc[0], inplace = True)
                        d1.drop(d1.index[0], inplace = True)
                        
                        dataf = pd.concat([dataf, d1])
                    
                # Process the data frame here 
                dataf[['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
                       'quarter', 'fiscal_year', 
                       'announce_datetime', 'currency', 'purl', 'quarter_end_date',
                       'created', 'updated', 'return_time']] = dataf[['event_id', 'company_id', 
                                                                      'stock_symbol',
                                                                      'isin', 'company_name',
                                                                      'quarter', 'fiscal_year',
                                                                      'announce_datetime',
                                                                      'currency', 'purl',
                                                                      'quarter_end_date',
                                                                      'created', 'updated',
                                                                      'return_time']].astype("string")
                # Datetime columns manipulate 
                dataf['fiscal_year'] = dataf['fiscal_year'].astype(int)
                if 'prelim_earnings_date' in dataf.columns:
                    dataf['prelim_earnings_date'] = dataf['prelim_earnings_date'].astype("string")
                    dataf['prelim_earnings_date'] = pd.to_datetime(dataf['prelim_earnings_date'])
                    
                if 'prelim_earnings_link' in dataf.columns:
                    dataf['prelim_earnings_link'] = dataf['prelim_earnings_link'].astype("string")
                    
                dataf['announce_datetime'] = pd.to_datetime(dataf['announce_datetime'])
                dataf['created'] = pd.to_datetime(dataf['created'])
                dataf['updated'] = pd.to_datetime(dataf['updated'])
                dataf['return_time'] = pd.to_datetime(dataf['return_time'])
                dataf['quarter_end_date'] = pd.to_datetime(dataf['quarter_end_date'])
                
                # Some numerics columns / may not be there sometimes 
                if 'amount_oc' in dataf.columns:
                    dataf['amount_oc'] = dataf['amount_oc'].astype(float)
                if 'prelim_amount_to' in dataf.columns:
                    dataf['prelim_amount_to'] = dataf['prelim_amount_to'].astype(float)
                if 'prelim_amount_from'in dataf.columns:
                    dataf['prelim_amount_from'] = dataf['prelim_amount_to'].astype(float)

                # Create asofdate (date) column
                dataf['date'] = start_dt
                dataf['date'] = pd.to_datetime(dataf['date'])
        
                dt_lbl = start_dt.strftime("%m-%d-%y")
                
                # Save to desired path (ToDo: Change path as needed!)
                print(dataf.columns)
                print(len(dataf))
                print(onedrive_pth + f"temp_{key}/{key}_webserv{dt_lbl}.parquet")
                dataf.to_parquet(onedrive_pth + f"temp_{key}/{key}_webserv{dt_lbl}.parquet")

            else:
                print("no data in this date range for ", evt, ", ",
                      from_dt, " to ", to_dt, "! :)")
        
    

https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=05/10/21&to=05/11/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'return_time', 'date'],
      dtype='object')
744
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv05-10-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=05/11/21&to=05/12/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'crea

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
125
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv05-24-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=05/25/21&to=05/26/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'return_time', 'date'],
    

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
68
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv06-08-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=06/09/21&to=06/10/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'prelim_earnings_date', 'prelim_earnings_link', 'date'],
      dtype='object')
73
C://Users/clj585/OneDrive - N

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
46
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv06-23-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=06/24/21&to=06/25/21&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
     

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'date'],
      dtype='object')
15
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv07-09-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=07/10/21&to=07/11/21&v=2&o=EVENTS
Are you here?
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'date'],
      dtype='object')
1
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv07-10-21.parquet
https://enchilada.wallstreethorizon.com/webserv

...

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'prelim_earnings_date', 'prelim_earnings_link', 'return_time',
       'prelim_amount_from', 'prelim_amount_to', 'date'],
      dtype='object')
538
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv02-28-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=03/01/22&to=03/02/22&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
    

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
      dtype='object')
285
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv03-15-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=03/16/22&to=03/17/22&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
    

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
      dtype='object')
347
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv03-29-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=03/30/22&to=03/31/22&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
    

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
106
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv04-13-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=04/14/22&to=04/15/22&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
    

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'created', 'updated', 'prelim_earnings_date',
       'prelim_amount_from', 'prelim_amount_to', 'prelim_earnings_link',
       'return_time', 'amount_oc', 'purl', 'date'],
      dtype='object')
1092
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps/eps_webserv04-27-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=04/28/22&to=04/29/22&v=2&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'created', 'updated', 'prelim_earnings_date',
       'prelim_amount_from', 'prelim_amount_to', 'prelim_earnings_link',
       'return_time', 'amount_oc', 'purl', 'date'],
   

In [2]:
# -*- coding: utf-8 -*-
"""
Redo the above gathering and pre-processing of parquet
files for the V3 API version of EPS event type. 

Some fields only returned when the param is specified
in the initial URL. Then parse these modified/expanded
XMLs outputted. 

"""

###################################################
# Loads packages 
###################################################

import urllib.request
import xmltodict
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time

# Define path here 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/"

###################################################
# Send Webservice Query & pull data 
###################################################

"""
Function: formURL

Returns URL generated for WSH Enchilada webservice query 

@params: 
   stock_sym_list: list of strings of stock symbols
   class: string or ideally only 1 string
   from_dt: start date formatted as string
   to_dt: end date of query as string
"""
def formURL(stock_sym_list, evtclass, from_dt, to_dt):
    
    home = "https://enchilada.wallstreethorizon.com/webservice6.asp"
    cust_id = "?c=NORTHWESTERN"
    passw = "&p=4IQD42P0"
    
    stock_syms = "&stock_symbols="+",".join(stock_sym_list)
    classes = "&classes="+evtclass
    
    fromdt = "&from=" + from_dt
    todt = "&to=" + to_dt
    
    v = "&v=3"
    opts = "&o=EVENTS"
    
    url_str = (home + cust_id + passw + 
               stock_syms + 
               classes + fromdt + todt + 
               v + opts)
    return url_str

"""
Function: perDelta

Returns an iterable with all the starting dates of time intervals.
Breaks the history time window up into 9-day windows. 

@params: 
   start: string formatted date
   end:also string formatted end date
   delta: time-skip needs to be in days
"""
def perDelta(start, end, delta):
    start = datetime.strptime(start, "%m/%d/%y")
    end = datetime.strptime(end, "%m/%d/%y")
    curr = start
    while curr < end:
        yield curr
        curr += delta

###################################################
# Process EPS announce date in general framework
###################################################

# PARAMETERS AND LISTS CAN BE MODIFIED HERE # 
stock_sym_list = ["*"]
# Entire list of desired event class types
class_list = ['EPS']

# ToDo Will: UPDATE BASED ON LOGS 
hist_from_dt = (datetime.today()-relativedelta(years=1)).strftime(
    "%m/%d/%y")
hist_to_dt = (datetime.today()).strftime("%m/%d/%y")

# The full list of V3 EPS columns in the XML output 
list_v3_cols = ['event_id', 'company_id', 
                'stock_symbol', 
                'isin', 'company_name',
                "announce_datetime", 
                "fiscal_year",
                "quarter", 
                'quarter_end_date',
                'currency',
                'amount_oc',
                'estimated_eps',
                'purl',
                'prelim_earnings_date',
                'prelim_amount_from',
                'prelim_amount_to',
                'prelim_earnings_link',
                'increase_decrease_code',
                'change_amount',
                'change_percent']

# The columns that should be strings
str_cols = ['event_id', 'company_name', 'stock_symbol', 'isin',
            'created', 'updated', 'return_time',
           'quarter', 'purl', 'prelim_earnings_link', 'currency',
           'increase_decrease_code']

# The columns that should be then converted into pandas datetime
dt_cols = ['announce_datetime', 'prelim_earnings_date', 'quarter_end_date',
          'created', 'updated', 'return_time']

# The columns that must be floats 
flt_cols = ['amount_oc', 'estimated_eps', 'prelim_amount_from',
           'prelim_amount_to', 'change_amount', 'change_percent']

# Columns that should be integers
int_cols = ['company_id', 'fiscal_year']
    
for evt in class_list:          # if stock wildcard only 1 evt at a time

    # Sleep condition avoid timeout
    if(class_list.index(evt)%4 == 0 and class_list.index(evt)!=0): 
        time.sleep(30)    
    
    # Generate 1-day intervals for query window
    for start_dt in perDelta(hist_from_dt, 
                            #(datetime.today()-relativedelta(months=1)).strftime("%m/%d/%y"),
                             hist_to_dt,
                             timedelta(days=1)): 
        
        # Formatting as strings 
        from_dt = start_dt.strftime("%m/%d/%y")
        to_dt = (start_dt + relativedelta(days=1)).strftime(
            "%m/%d/%y")
        
        # Formulate URL for webservice query here 
        url3 = formURL(stock_sym_list, evt, 
                       from_dt, to_dt)
        print(url3)

        # Send, process request and save tree structure 
        rsp = urllib.request.urlopen(url3)
        string = rsp.read().decode('utf-8')
    
        # Conversion to dictionary 
        dictdata = xmltodict.parse(string)

        # For each dictionary list (keyed by event class) get data 
        for key in dictdata["webservice6"].keys():
            dataf = pd.DataFrame()                  # placeholder
            
            if key != "error":
                # Check if OrderedDict or list
                if(type(dictdata["webservice6"][key]) != list):
                    print("Are you here?")
                    dct = dictdata["webservice6"][key]
                
                    # Save to data frame and reformat
                    d1 = pd.DataFrame(dct.items(), columns=None)
                    d1 = d1.T
                    d1.rename(columns=d1.iloc[0], inplace = True)
                    d1.drop(d1.index[0], inplace = True)
                    dataf = pd.concat([dataf, d1])
                
                else:
                    print("It will be a list!")
                    for i in range(len(dictdata["webservice6"][key])):
                
                        # Number of dictionaries equal to number of event id's 
                        dct = dictdata["webservice6"][key][i]
                
                        # Save to data frame and reformat
                        d1 = pd.DataFrame(dct.items(), columns=None)
                        d1 = d1.T
                        d1.rename(columns=d1.iloc[0], inplace = True)
                        d1.drop(d1.index[0], inplace = True)
                        
                        dataf = pd.concat([dataf, d1])
                    
                # Process the data frame here - new procedure 
                for col in dataf.columns:
                    if col in str_cols:
                        dataf[col] = dataf[col].astype("string")
                    if col in dt_cols:
                        dataf[col] = pd.to_datetime(dataf[col])
                    if col in int_cols:
                        dataf[col] = dataf[col].astype(int)
                    if col in flt_cols:
                        if col == "change_percent":
                            dataf[col] = dataf[col].replace(",", "", regex=True)
                            dataf[col] = dataf[col].replace("%", "", regex=True).astype(float)
                        else:
                            dataf[col] = dataf[col].astype(float)
                    
                # Create asofdate (date) column
                dataf['date'] = start_dt
                dataf['date'] = pd.to_datetime(dataf['date'])
        
                dt_lbl = start_dt.strftime("%m-%d-%y")
                
                # Save to desired path (ToDo: Change path as needed!)
                print(dataf.columns)
                print(len(dataf))
                print(onedrive_pth + f"temp_{key}_V3/{key}_webserv{dt_lbl}.parquet")
                dataf.to_parquet(onedrive_pth + f"temp_{key}_V3/{key}_webserv_V3_{dt_lbl}.parquet")

            else:
                print("no data in this date range for ", evt, ", ",
                      from_dt, " to ", to_dt, "! :)")
    

https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/03/21&to=10/04/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent', 'date'],
      dtype='object')
7
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv10-03-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/04/21&to=10/05/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
   

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
32
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv10-15-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/16/21&to=10/17/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_perc

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'return_time', 'prelim_earnings_date', 'prelim_amount_from',
       'prelim_amount_to', 'prelim_earnings_link', 'date'],
      dtype='object')
1075
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv10-27-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/28/21&to=10/29/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_pe

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'prelim_earnings_link', 'prelim_amount_from',
       'prelim_amount_to', 'date'],
      dtype='object')
881
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv11-08-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=11/09/21&to=11/10/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_per

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'date'],
      dtype='object')
65
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv11-21-21.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=11/22/21&to=11/23/21&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'date'],
      dtype=

...

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'prelim_earnings_link', 'date'],
      dtype='object')
941
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv08-02-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=08/03/22&to=08/04/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'return_time', 'prelim_earnings_date', 'prelim_earnings_link',
       'prelim_amount_from', 'prelim_amount_to', 'date'],
      dtype='object')
500
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv08-15-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=08/16/22&to=08/17/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amoun

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'prelim_earnings_link', 'date'],
      dtype='object')
92
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv08-28-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=08/29/22&to=08/30/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent', 'date'],
      dtype='object')
54
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv09-12-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=09/13/22&to=09/14/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date', 'prelim_amount_from', 'prelim_amount_to',
       'pr

It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'return_time', 'prelim_earnings_date', 'prelim_earnings_link', 'date'],
      dtype='object')
35
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv09-26-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=09/27/22&to=09/28/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'amount_oc', 'purl', 'created', 'updated', 'return_time',
       'increase_decrease_code', 'change_amount', 'change_percent',
       'prelim_earnings_date

In [30]:
dataf.head()

,event_id,company_id,stock_symbol,isin,company_name,announce_datetime,fiscal_year,quarter,quarter_end_date,currency,...,updated,prelim_earnings_date,prelim_amount_from,prelim_amount_to,prelim_earnings_link,increase_decrease_code,change_amount,change_percent,return_time,date
1,4PUM1F46,1431663281,NXGN,US65343C1027,"NextGen Healthcare, Inc.",2022-05-17 16:05:00,2022,Q4,2022-03-31,USD,...,2022-05-17 16:13:11,2022-05-02 16:10:00,0.18,0.2,https://www.businesswire.com/news/home/2022050...,D,-0.05,-20.8,2022-06-02 09:15:36,2022-05-02
1,4PWS17LA,1431692326,8031.JP,JP3893600001,Mitsui,2022-05-02 00:00:00,2022,Q4,2022-03-31,JPY,...,2022-05-02 06:47:39,NaT,NaN,NaN,<NA>,<NA>,NaN,NaN,2022-06-02 09:15:36,2022-05-02
1,4PX315SF,1431691805,RATOB.SE,SE0000111940,Ratos AB,2022-05-02 00:00:00,2022,Q1,2022-03-31,SEK,...,2022-05-02 07:05:08,NaT,NaN,NaN,<NA>,D,-1.60,-170.2,2022-06-02 09:15:36,2022-05-02
1,4PZH18D0,1431661768,ENIA,US29274F1049,Enel Americas SA,2022-05-02 00:00:00,2022,Q1,2022-03-31,USD,...,2022-05-02 13:46:57,NaT,NaN,NaN,<NA>,<NA>,NaN,NaN,2022-06-02 09:15:36,2022-05-02
1,4PX41P28,1431694596,ALICORI1.PE,PEP214001005,Alicorp SA,2022-05-02 00:00:00,2022,Q1,2022-03-31,PEN,...,2022-05-03 07:49:33,NaT,NaN,NaN,<NA>,<NA>,NaN,NaN,2022-06-02 09:15:36,2022-05-02


In [31]:
dataf.dtypes

event_id                          string
company_id                         int32
stock_symbol                      string
isin                              string
company_name                      string
announce_datetime         datetime64[ns]
fiscal_year                        int32
quarter                           string
quarter_end_date          datetime64[ns]
currency                          string
amount_oc                        float64
purl                              string
created                   datetime64[ns]
updated                   datetime64[ns]
prelim_earnings_date      datetime64[ns]
prelim_amount_from               float64
prelim_amount_to                 float64
prelim_earnings_link              string
increase_decrease_code            string
change_amount                    float64
change_percent                   float64
return_time               datetime64[ns]
date                      datetime64[ns]
dtype: object

In [3]:
dataf

,event_id,company_id,stock_symbol,isin,company_name,announce_datetime,fiscal_year,quarter,quarter_end_date,currency,...,updated,return_time,amount_oc,increase_decrease_code,change_amount,change_percent,estimated_eps,prelim_earnings_date,prelim_earnings_link,date
1,4QAP2OG9,1431700497,3186.JP,JP3758210003,NEXTAGE,2022-10-03 00:00:00,2022,Q3,2022-08-31,JPY,...,2022-10-03 06:34:40,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4QAM2P83,1431699533,7965.JP,JP3437400009,Zojirushi,2022-10-03 00:00:00,2022,Q3,2022-08-20,JPY,...,2022-10-03 06:38:45,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4Q8K2T5R,1431691838,8227.JP,JP3358200008,SHIMAMURA Co. Ltd.,2022-10-03 00:00:00,2022,Q2,2022-08-20,JPY,...,2022-10-03 06:40:34,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4QAL2WY0,1431695716,9793.JP,JP3485600005,Daiseki Co. Ltd.,2022-10-03 00:00:00,2023,Q2,2022-08-31,JPY,...,2022-10-03 06:42:45,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4PZO182F,1431690790,ARYN.CH,CH0043238366,ARYZTA AG,2022-10-03 00:00:00,2022,H2,2022-07-31,EUR,...,2022-10-03 06:43:13,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4PZL10ID,1431693328,BON.FR,FR0000063935,Bonduelle SA,2022-10-03 01:00:00,2022,H2,2022-06-30,EUR,...,2022-10-03 06:30:39,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,NaN,NaT,<NA>,2022-10-02
1,4QAW2T5Z,1431682557,CSVI,US20539A1051,Computer Services Inc.,2022-10-03 08:00:00,2023,Q2,2022-08-31,USD,...,2022-10-03 08:09:51,2022-10-03 11:53:11,0.45,D,-0.09,-16.7,NaN,NaT,<NA>,2022-10-02
1,4Q7U2WZ5,1431696931,LEGN,US52490G1022,Legend Biotech Corporation,2022-10-03 16:00:00,2022,Q2,2022-06-30,<NA>,...,2022-09-26 18:29:44,2022-10-03 11:53:11,NaN,<NA>,NaN,NaN,-0.575,2022-08-05,https://www.businesswire.com/news/home/2022080...,2022-10-02


In [4]:
# -*- coding: utf-8 -*-
"""
Redo the above gathering and pre-processing of parquet
files for the V3 API version of EPS event type. 

Some fields only returned when the param is specified
in the initial URL. Then parse these modified/expanded
XMLs outputted. 

"""

###################################################
# Loads packages 
###################################################

import urllib.request
import xmltodict
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time

# Define path here 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/"

###################################################
# Send Webservice Query & pull data 
###################################################

"""
Function: formURL

Returns URL generated for WSH Enchilada webservice query 

@params: 
   stock_sym_list: list of strings of stock symbols
   class: string or ideally only 1 string
   from_dt: start date formatted as string
   to_dt: end date of query as string
"""
def formURL(stock_sym_list, evtclass, from_dt, to_dt):
    
    home = "https://enchilada.wallstreethorizon.com/webservice6.asp"
    cust_id = "?c=NORTHWESTERN"
    passw = "&p=4IQD42P0"
    
    stock_syms = "&stock_symbols="+",".join(stock_sym_list)
    classes = "&classes="+evtclass
    
    fromdt = "&from=" + from_dt
    todt = "&to=" + to_dt
    
    v = "&v=3"
    opts = "&o=EVENTS"
    
    url_str = (home + cust_id + passw + 
               stock_syms + 
               classes + fromdt + todt + 
               v + opts)
    return url_str

"""
Function: perDelta

Returns an iterable with all the starting dates of time intervals.
Breaks the history time window up into 9-day windows. 

@params: 
   start: string formatted date
   end:also string formatted end date
   delta: time-skip needs to be in days
"""
def perDelta(start, end, delta):
    start = datetime.strptime(start, "%m/%d/%y")
    end = datetime.strptime(end, "%m/%d/%y")
    curr = start
    while curr < end:
        yield curr
        curr += delta

###################################################
# Process EPS announce date in general framework
###################################################

# PARAMETERS AND LISTS CAN BE MODIFIED HERE # 
stock_sym_list = ["*"]
# Entire list of desired event class types
class_list = ['EPS']

# ToDo Will: UPDATE BASED ON LOGS 
hist_from_dt = (datetime.today()-relativedelta(days=4)).strftime("%m/%d/%y")
#(datetime.today()-relativedelta(years=1)).strftime(
    #"%m/%d/%y")
hist_to_dt = (datetime.today()).strftime("%m/%d/%y")

# The full list of V3 EPS columns in the XML output 
list_v3_cols = ['event_id', 'company_id', 
                'stock_symbol', 
                'isin', 'company_name',
                "announce_datetime", 
                "fiscal_year",
                "quarter", 
                'quarter_end_date',
                'currency',
                'amount_oc',
                'estimated_eps',
                'purl',
                'prelim_earnings_date',
                'prelim_amount_from',
                'prelim_amount_to',
                'prelim_earnings_link',
                'increase_decrease_code',
                'change_amount',
                'change_percent']

# The columns that should be strings
str_cols = ['event_id', 'company_name', 'stock_symbol', 'isin',
            'created', 'updated', 'return_time',
           'quarter', 'purl', 'prelim_earnings_link', 'currency',
           'increase_decrease_code']

# The columns that should be then converted into pandas datetime
dt_cols = ['announce_datetime', 'prelim_earnings_date', 'quarter_end_date',
          'created', 'updated', 'return_time']

# The columns that must be floats 
flt_cols = ['amount_oc', 'estimated_eps', 'prelim_amount_from',
           'prelim_amount_to', 'change_amount', 'change_percent']

# Columns that should be integers
int_cols = ['company_id', 'fiscal_year']
    
for evt in class_list:          # if stock wildcard only 1 evt at a time

    # Sleep condition avoid timeout
    if(class_list.index(evt)%4 == 0 and class_list.index(evt)!=0): 
        time.sleep(30)    
    
    # Generate 1-day intervals for query window
    for start_dt in perDelta(hist_from_dt, 
                            #(datetime.today()-relativedelta(months=1)).strftime("%m/%d/%y"),
                             hist_to_dt,
                             timedelta(days=1)): 
        
        # Formatting as strings 
        from_dt = start_dt.strftime("%m/%d/%y")
        to_dt = (start_dt + relativedelta(days=1)).strftime(
            "%m/%d/%y")
        
        # Formulate URL for webservice query here 
        url3 = formURL(stock_sym_list, evt, 
                       from_dt, to_dt)
        print(url3)

        # Send, process request and save tree structure 
        rsp = urllib.request.urlopen(url3)
        string = rsp.read().decode('utf-8')
    
        # Conversion to dictionary 
        dictdata = xmltodict.parse(string)

        # For each dictionary list (keyed by event class) get data 
        for key in dictdata["webservice6"].keys():
            dataf = pd.DataFrame()                  # placeholder
            
            if key != "error":
                # Check if OrderedDict or list
                if(type(dictdata["webservice6"][key]) != list):
                    print("Are you here?")
                    dct = dictdata["webservice6"][key]
                
                    # Save to data frame and reformat
                    d1 = pd.DataFrame(dct.items(), columns=None)
                    d1 = d1.T
                    d1.rename(columns=d1.iloc[0], inplace = True)
                    d1.drop(d1.index[0], inplace = True)
                    dataf = pd.concat([dataf, d1])
                
                else:
                    print("It will be a list!")
                    for i in range(len(dictdata["webservice6"][key])):
                
                        # Number of dictionaries equal to number of event id's 
                        dct = dictdata["webservice6"][key][i]
                
                        # Save to data frame and reformat
                        d1 = pd.DataFrame(dct.items(), columns=None)
                        d1 = d1.T
                        d1.rename(columns=d1.iloc[0], inplace = True)
                        d1.drop(d1.index[0], inplace = True)
                        
                        dataf = pd.concat([dataf, d1])
                    
                # Process the data frame here - new procedure 
                for col in dataf.columns:
                    if col in str_cols:
                        dataf[col] = dataf[col].astype("string")
                    if col in dt_cols:
                        dataf[col] = pd.to_datetime(dataf[col])
                    if col in int_cols:
                        dataf[col] = dataf[col].astype(int)
                    if col in flt_cols:
                        if col == "change_percent":
                            dataf[col] = dataf[col].replace(",", "", regex=True)
                            dataf[col] = dataf[col].replace("%", "", regex=True).astype(float)
                        else:
                            dataf[col] = dataf[col].astype(float)
                    
                # Create asofdate (date) column
                dataf['date'] = start_dt
                dataf['date'] = pd.to_datetime(dataf['date'])
        
                dt_lbl = start_dt.strftime("%m-%d-%y")
                
                # Save to desired path (ToDo: Change path as needed!)
                print(dataf.columns)
                print(len(dataf))
                print(onedrive_pth + f"temp_{key}_V3/{key}_webserv{dt_lbl}.parquet")
                dataf.to_parquet(onedrive_pth + f"temp_{key}_V3/{key}_webserv_V3_{dt_lbl}.parquet")

            else:
                print("no data in this date range for ", evt, ", ",
                      from_dt, " to ", to_dt, "! :)")
    

https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=09/30/22&to=10/01/22&v=3&o=EVENTS
It will be a list!
Index(['event_id', 'company_id', 'stock_symbol', 'isin', 'company_name',
       'announce_datetime', 'fiscal_year', 'quarter', 'quarter_end_date',
       'currency', 'purl', 'created', 'updated', 'return_time', 'amount_oc',
       'increase_decrease_code', 'change_amount', 'change_percent', 'date'],
      dtype='object')
25
C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/Data/temp_eps_V3/eps_webserv09-30-22.parquet
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/01/22&to=10/02/22&v=3&o=EVENTS
no data in this date range for  EPS ,  10/01/22  to  10/02/22 ! :)
https://enchilada.wallstreethorizon.com/webservice6.asp?c=NORTHWESTERN&p=4IQD42P0&stock_symbols=*&classes=EPS&from=10/02/22&to=10/03/22&v=3&o=EVENTS
It will be a list!
Index(['ev